In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential      #线性模型框架
from keras.layers import Dense, Dropout, Flatten   #Dense全连接层; Flatten平铺层
from keras.layers import Conv2D, MaxPooling2D   #Conv2D 二维卷积层; MaxPooling2D: 二维池化层

# 1. 准备数据:

In [2]:
(x_train_0, y_train_0), (x_test_0, y_test_0) = mnist.load_data()

In [3]:
n_train=x_train_0.shape[0]
n_test=x_test_0.shape[0]
width,height=x_train_0.shape[1], x_train_0.shape[2]

In [4]:
x_train=x_train_0.reshape(n_train, width, height, 1)
x_test=x_test_0.reshape(n_test, width, height, 1)
print(x_train_0.shape)
print(x_train.shape)

(60000, 28, 28)
(60000, 28, 28, 1)


In [5]:
from keras.utils import np_utils
y_train=np_utils.to_categorical(y_train_0)
y_test=np_utils.to_categorical(y_test_0)

In [6]:
print(y_train.shape)
print(y_train[0])
print(y_train_0.shape)
print("处理前的最大值为%f" % x_train.max())

(60000, 10)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
(60000,)
处理前的最大值为255.000000


In [7]:
x_train=x_train/255
x_test=x_test/255
print("处理后的最大值为%f" % x_train.max())

处理后的最大值为1.000000


# 2. 构造模型:

In [8]:
#keras中的线性模型
model=Sequential()  
#二维卷积层, 32个5X5的卷积核, 使用ReLU作为激活函数
model.add(Conv2D(32,(5,5),activation="relu",input_shape=(width,height,1))) 
#最大池化层: 2X2大小的池化核
model.add(MaxPooling2D(pool_size=(2,2))) 
#二维卷积层: 32个3x3的卷积核,使用ReLU作为激活函数
model.add(Conv2D(32, (3,3), activation="relu"))  
#最大池化层: 2X2大小的池化核
model.add(MaxPooling2D(pool_size=(2,2)))  
#平铺层: 将数据形状转为向量
model.add(Flatten())
#全连接层: 隐藏层维度为256, 使用ReLU作为激活函数
model.add(Dense(256, activation="relu"))
#全连接层: 隐藏层维度为10, 使用Softmax作为激活函数, 输出每个分类的概率
model.add(Dense(10,activation="softmax"))


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               205056    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2

In [10]:
#优化器
#Adam优化器, 损失函数:分类交叉熵
#度量指标: 准确率
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

# 3. 开始训练:

In [ ]:
#开始训练
import time
start_time=time.time()
print("pre fit:" + str(start_time))
# time.sleep(2)
model.fit(x_train, y_train, batch_size=24, epochs=5, validation_data=(x_test, y_test))
end_time=time.time()
print("fit done:"+str(end_time))
print("duration:" + str(end_time-start_time))

pre fit:1634033854.3968427
Epoch 1/5
2500/2500 [==============================] - 64s 25ms/step - loss: 0.1141 - accuracy: 0.9646 - val_loss: 0.0414 - val_accuracy: 0.9868
Epoch 2/5
2500/2500 [==============================] - 43s 17ms/step - loss: 0.0413 - accuracy: 0.9872 - val_loss: 0.0307 - val_accuracy: 0.9893
Epoch 3/5
2500/2500 [==============================] - 46s 18ms/step - loss: 0.0290 - accuracy: 0.9908 - val_loss: 0.0266 - val_accuracy: 0.9912
Epoch 4/5
2500/2500 [==============================] - 37s 15ms/step - loss: 0.0221 - accuracy: 0.9932 - val_loss: 0.0319 - val_accuracy: 0.9898
Epoch 5/5
1501/2500 [=================>............] - ETA: 13s - loss: 0.0166 - accuracy: 0.9948- ETA: 13s - loss: 0.0163 - accuracy:

In [ ]:
score = model.evaluate(x_test, y_test)
print("损失为%f" % score[0])
print("准确度为%f" % score[1])

In [ ]:


print(x_test[0:1])
model.predict(x_test[0:1])


# 4. 预测:

In [ ]:
import matplotlib
import matplotlib.image as img

# import glob
from PIL import Image
import numpy as np

input_path="test/5.jpg"
# input_path="test/3_out_sm.jpg"
# input_path="test/3_out_gray.bmp"
im=Image.open(input_path)
print(im.size)
im=im.resize((28,28))  # 转换成28X28的尺寸
print(im.size)

im_1=im.convert("L")
im_data=im_1.getdata()
im_array = np.array(list(im_data))
im_array= im_array.reshape(28,28)
im_array=255-im_array
im_array=im_array.reshape(1,28,28,1)

predict_result = model.predict(im_array)
print("识别结果:%d" % np.argmax(predict_result))